In [ ]:
import os
import ahpy
import json
import math
import random
import hashlib
import jenkspy
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt

from utils import Infiltrometro, ALL_FUNCTIONS, nse, points_distance, USO_SOLO_CLASS, SOIL_TYPES, USO_SOLO_MAPBIOMAS

from shapely import Point, Polygon, box
from tqdm import tqdm
from concurrent import futures
from xgboost import XGBRegressor
from shapely.geometry import Point
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from itertools import combinations, product, islice
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import QuantileTransformer

# Tabela com os pesos para cada tipo de dado

In [ ]:
tabela_pesos = pd.DataFrame({
    "Fator":["Form. Geo.", "Uso Solo", "Declividade", "Elevação", "Textura"],
    "Influência AHP":[0.419, 0.263, 0.160, 0.097, 0.062],
    "Influência MIF":[0.278, 0.222, 0.167, 0.167, 0.167],
})
tabela_pesos

In [ ]:
# Uso do solo para influência na infiltração
uso_solo_tif = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\UsoSoloMG.tif")
uso_solo_tif = uso_solo_tif.rio.reproject("EPSG:31983")

uso_solo_index = np.zeros_like(uso_solo_tif, dtype=float)

for tipo, values in tqdm(USO_SOLO_MAPBIOMAS.items(), "Aplicando tipo de uso do solo", len(USO_SOLO_MAPBIOMAS)):
    uso_solo_index[uso_solo_tif.values == tipo] = values['infiltration_index']

uso_solo_index[uso_solo_tif.values < 0] = np.nan

del uso_solo_tif, tipo, values
uso_solo_index

In [ ]:
form_geo_df  = gpd.read_file(r"D:\Mestrado\Trabalho Final\SIG\Unidade Geológicas.zip")
form_geo_df.to_crs("EPSG:31983", inplace=True)
form_geo_index = form_geo_df[['class_infi', 'geometry']]
del form_geo_df
form_geo_index


In [ ]:
a = 1 + "a" #<- Obter a elevação e aplicar o Jenks
elevacao_tif = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\Elevation.tif") # <- Obter a elevação para MG

elevation_index = np.zeros_like(elevacao_tif.values, dtype=float)
elevation_index[(elevation_index < 800)] = 9
elevation_index[(elevation_index >= 800) & (elevation_index < 900)] = 8
elevation_index[(elevation_index >= 900) & (elevation_index < 1000)] = 7
elevation_index[(elevation_index >= 1000) & (elevation_index < 1100)] = 6
elevation_index[(elevation_index >= 1100) & (elevation_index < 1200)] = 5
elevation_index[(elevation_index >= 1200) & (elevation_index < 1300)] = 4
elevation_index[(elevation_index >= 1300) & (elevation_index < 1400)] = 3
elevation_index[(elevation_index >= 1400) & (elevation_index < 1500)] = 2
elevation_index[(elevation_index >= 1500)] = 1
elevation_index[elevacao_tif.values < 0] = np.nan

del elevacao_tif
elevation_index

In [ ]:
# Declividade é similar a rugosidade, pois quanto maior for a declividade menor é a infiltração
a = 1 + "A" #<- Obter a declividade e aplicar o Jenks
declive_tif  = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\Slope.tif")     # <- Obter a declivid para MG

slope_index = np.zeros_like(declive_tif.values, dtype=float)

slope_index[(slope_index < 8)]                        = 4
slope_index[(slope_index >= 8)  & (slope_index < 15)] = 3
slope_index[(slope_index >= 15) & (slope_index < 25)] = 2
slope_index[(slope_index >= 25)]                      = 1
slope_index[declive_tif.values < 0] = np.nan

del declive_tif
slope_index

In [ ]:
textura_df = gpd.read_file(r"D:\Mestrado\Trabalho Final\SIG\Tipos de Solo IDE.zip")
textura_df.to_crs("EPSG:31983", inplace=True)
texture_index = textura_df[['class_infi', 'geometry']]
del form_geo_df
texture_index



# Cálculos dos valores de Potencial de infiltração para cada um dos métodos

## Método de Jenks

In [ ]:
for tipo in ["MIF", "AHP"]:
    potencial_infiltracao = (
        tabela_pesos.loc[1][f"Influência {tipo}"] * uso_solo_index + \
        tabela_pesos.loc[2][f"Influência {tipo}"] * slope_index + \
        tabela_pesos.loc[3][f"Influência {tipo}"] * elevation_index
    )

    for idx, row in form_geo_index.iterrows():
        form_geo_val = row.class_infi
        geom:Polygon = row.geometry

        mask = geom.contains(texture_index.geometry.values)
        for idx, row in texture_index[mask]:
            texture_val = row.class_infi
            geom: Polygon = row.geometry

            bbox = box(*geom.bounds)

            Criar BBOX, filtrar rester pelo BBOX
            Filtrar Geometrias dentro da outra
    
    potencial_infiltracao = (
        tabela_pesos.loc[0][f"Influência {tipo}"] * form_geo_index + \
        tabela_pesos.loc[4][f"Influência {tipo}"] * texture_index
    )

    potencial_infiltracao[np.isnan(potencial_infiltracao)] = -9999

    # Garantir que o resultado tem o mesmo shape que o raster base
    potencial_infiltracao = potencial_infiltracao.reshape(textura.shape[1:])

    # Criar um novo DataArray com as mesmas coordenadas e metadados
    potencial_da = xr.DataArray(
        potencial_infiltracao.astype("float32"),
        dims=("y", "x"),
        coords={"x": textura.x, "y": textura.y},
        name=f"potencial_infiltracao_{tipo.lower()}"
    )

    # Copiar CRS e transformar em um raster compatível
    potencial_da = potencial_da.rio.write_crs(textura.rio.crs)
    potencial_da = potencial_da.rio.reproject_match(textura)

    # (opcional) definir valor nodata
    potencial_da = potencial_da.rio.write_nodata(-9999)

    # Salvar como GeoTIFF
    saida = fr"D:\Mestrado\Trabalho Final\SIG\Potencial_Infiltracao_{tipo}_{metodo}.tif"
    potencial_da.rio.to_raster(saida)
    print(f"✅ Raster salvo com sucesso em: '{saida}'")

    
    # Classificação utilizando Jenks
    k = 5

    flat = potencial_infiltracao.flatten()
    flat = flat[flat>=0]

    breaks = jenkspy.jenks_breaks(np.random.choice(flat, size=100_000, replace=False), n_classes=k)
    print(f"Tipo: {tipo}\nMétodo: {metodo}\nClasses Jenks:\n\t{'\n\t'.join([str(i) for i in breaks])}")

    classificacao_potencial = np.full_like(potencial_infiltracao, fill_value=-9999)

    last_break = None
    now_break = None
    for i, _break in enumerate(breaks, 1):
        now_break = _break

        mask = potencial_infiltracao <= now_break

        if last_break is not None:
            mask = mask & (potencial_infiltracao > last_break)
        last_break = now_break

        classificacao_potencial[mask] = i
    
    maskNone = potencial_infiltracao < 0
    classificacao_potencial[maskNone] = -9999

    # Criar um novo DataArray com as mesmas coordenadas e metadados
    potencial_de = xr.DataArray(
        classificacao_potencial.astype("float32"),
        dims=("y", "x"),
        coords={"x": textura.x, "y": textura.y},
        name=f"classificacao_potencial_{tipo.lower()}"
    )

    # Copiar CRS e transformar em um raster compatível
    potencial_de = potencial_de.rio.write_crs(textura.rio.crs)
    potencial_de = potencial_de.rio.reproject_match(textura)

    # (opcional) definir valor nodata
    potencial_de = potencial_de.rio.write_nodata(-9999)

    # Salvar como GeoTIFF
    saida = fr"D:\Mestrado\Trabalho Final\SIG\Classificacao_Infiltracao_{tipo}_{metodo}.tif"
    potencial_de.rio.to_raster(saida)
    print(f"✅ Raster salvo com sucesso em: '{saida}'")

# Classificação com Jenks para a condutividade observada

In [ ]:
K = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\CondutividadeHidraulica.tif")

# Classificação utilizando Jenks
k = 5

values = K.values[0]
flat = values.flatten()
flat = flat[flat>=0]

breaks = jenkspy.jenks_breaks(np.random.choice(flat, size=100_000, replace=False), n_classes=k)
print(f"Condutividade:\nClasses Jenks:\n\t{'\n\t'.join([str(i) for i in breaks])}")

classificacao_condutividade = np.full_like(values, fill_value=-9999)

last_break = None
now_break = None
for i, _break in enumerate(breaks, 1):
    now_break = _break

    mask = values <= now_break

    if last_break is not None:
        mask = mask & (values > last_break)
    last_break = now_break

    classificacao_condutividade[mask] = i

maskNone = values < 0
classificacao_condutividade[maskNone] = -9999

# Criar um novo DataArray com as mesmas coordenadas e metadados
potencial_de = xr.DataArray(
    classificacao_condutividade.astype("float32"),
    dims=("y", "x"),
    coords={"x": K.x, "y": K.y},
    name=f"classificacao_condutividade"
)

# Copiar CRS e transformar em um raster compatível
potencial_de = potencial_de.rio.write_crs(K.rio.crs)
potencial_de = potencial_de.rio.reproject_match(K)

# (opcional) definir valor nodata
potencial_de = potencial_de.rio.write_nodata(-9999)

# Salvar como GeoTIFF
saida = fr"D:\Mestrado\Trabalho Final\SIG\ClassificacaoCondutividadeHidraulica.tif"
potencial_de.rio.to_raster(saida)
print(f"✅ Raster salvo com sucesso em: '{saida}'")

In [ ]:
# Jenks da classificação para adicionar na legenda do mapa
flat = classificacao_condutividade.flatten()
flat = flat[flat>=0]

breaks = jenkspy.jenks_breaks(np.random.choice(flat, size=100_000, replace=False), n_classes=k)
print(f"Condutividade Legenda:\nClasses Jenks:\n\t{'\n\t'.join([str(i) for i in breaks])}")